In [1]:
import numpy as np
import pandas as pd
import transformers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
import torch
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification

d:\GITHUB\AXTR LABS\Fine_Tune\gpu_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("train.csv")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
# To train the CustomModel you fine tuned.

model_path = "CustomModel"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [4]:
model = model.to("cuda") if torch.cuda.is_available() else model.to("cpu")

In [5]:
data = data[["comment_text","toxic"]]
data = data[0:1000]

In [6]:
X = list(data["comment_text"])
y = list(data["toxic"])
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, truncation=True, padding=True, max_length=512)
X_val_tokenized = tokenizer(X_val, truncation=True, padding=True, max_length=512)

In [7]:
print(len(X_train), len(X_val))
print(len(y_train), len(y_val))

800 200
800 200


In [8]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels=None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, index):
    item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
    if self.labels:
      item["labels"] = torch.tensor(self.labels[index])
    return item

  def __len__(self):
    return len(self.encodings["input_ids"])

In [9]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [10]:
def compute_metrics(p):
  pred, labels = p
  pred = np.argmax(pred, axis=1)

  accuracy = accuracy_score(y_true=labels, y_pred=pred)
  recall = recall_score(y_true=labels, y_pred=pred)
  precision = precision_score(y_true=labels, y_pred=pred)
  f1 = f1_score(y_true=labels,y_pred=pred)

  return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [11]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=3,
    per_device_train_batch_size=8
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [12]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=300, training_loss=0.06008340199788412, metrics={'train_runtime': 167.9925, 'train_samples_per_second': 14.286, 'train_steps_per_second': 1.786, 'total_flos': 631466532864000.0, 'train_loss': 0.06008340199788412, 'epoch': 3.0})

In [13]:
trainer.evaluate()

{'eval_loss': 0.21029826998710632,
 'eval_accuracy': 0.96,
 'eval_recall': 0.7619047619047619,
 'eval_precision': 0.8421052631578947,
 'eval_f1': 0.8,
 'eval_runtime': 3.4724,
 'eval_samples_per_second': 57.598,
 'eval_steps_per_second': 7.2,
 'epoch': 3.0}

In [14]:
text = "That point was good"

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize input and move to the same device
inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
inputs = {key: val.to(device) for key, val in inputs.items()}

# Forward pass
outputs = model(**inputs)

# Get predictions
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()

print(predictions)

[[9.9974114e-01 2.5886143e-04]]


In [15]:
np.set_printoptions(suppress=True)

In [16]:
trainer.save_model("CustomModel")